This notebook discusses the meaning of `approxChol()` and `LDLinv`. In summary, `approxChol()` takes the adjacency matrix `a` of a graph, and output an appoximate inverse of the laplacian. That inverse is represented by `LDLinv` structure, which can be applied to vectors using `LDLsolver()`

In [31]:
using LinearAlgebra
using SparseArrays
using Statistics
import Laplacians as Lap
using CUDA

Helper functions. `getLDLi(mat)` returns the `LDLinv` structure for the Laplacian of `mat`. `applyLDLi(ldli, mat)` applies `LDLinv` to a matrix `mat`.

In [10]:
getLDLi(mat) = Lap.approxChol(Lap.LLmatp(mat))
applyLDLi(ldli, mat) = sparse(reduce(hcat, map(x -> Lap.LDLsolver(ldli, Vector(x)), eachcol(mat))))

applyLDLi (generic function with 1 method)

Generate a random graph with 10000 nodes as `a`. Calculate its Laplacian as `la`. Get its `LDLinv` structure.

In [21]:
a = Lap.pure_random_graph(10000)
la = Lap.lap(a)
ldli = getLDLi(a)
a

10000×10000 SparseArrays.SparseMatrixCSC{Float64, Int64} with 80000 stored entries:
⠻⣦⡀⠀⠳⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⣄⠀⠈
⠀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⣄
⢽⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻
⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣷⣄⠀
⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠈⠻⣗
⠙⢿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⡀⠈⠻⣦⡀⠀
⡀⠀⠙⢿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⢦⠀⠈⠻⣦

Applying `LDLinv` to the graph laplacian will transform it to identity matrix. Thus, the linear operator represented by `LDLinv` is the approximate inverse of the graph laplcain.

In [34]:
res = applyLDLi(ldli, la)
println("the mean of deviation from identity matrix for all entries is $(mean(res - 1.0I))")
println("the upper left corner of the resulting matrix is: ")
res[1:15, 1:15]

the mean of deviation from identity matrix for all entries is -0.0001
the upper left corner of the resulting matrix is: 


15×15 SparseMatrixCSC{Float64, Int64} with 225 stored entries:
  1.14306      0.0126771     0.0172945    …  -0.0253597     0.00653636
  0.0123504    1.16515       0.0503812       -0.00289762    0.00169255
 -0.0024889    0.0466095     1.11742          0.000808095  -0.000301933
 -0.00352406   0.0133843    -0.000792009      0.000986136  -0.00142257
 -0.00171223   0.00160572   -0.022854         0.000337445   0.000173126
 -0.00137628   0.00148531   -0.00633946   …   3.06648e-5    7.53624e-5
 -0.00117139   0.00106028   -0.00447675      -0.000157608  -5.75672e-5
 -0.000841139  0.000231387  -0.0015564       -0.000311532  -0.000213066
 -0.000743585  0.000202244  -0.000696326     -0.00049275   -0.000330041
 -0.000945985  0.000268603  -0.000305977     -0.000982721  -0.000513379
 -0.00124893   0.000336132   2.64695e-5   …  -0.000964667  -0.00115675
 -0.00298721   3.72796e-6    0.000380135     -0.003502     -0.00407727
 -0.00603306   0.00044673    0.00137892       0.00224336   -0.0256941
 -0.020640